In [17]:
%matplotlib inline
import matplotlib.image as mpimg
import numpy as np
import matplotlib.pyplot as plt
import os,sys
from PIL import Image
from torch.utils.data import Dataset, random_split
import torchvision.io as io
from torchvision import transforms 
import torch
import torchvision
from torch import nn

In [18]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [19]:
if "google.colab" in sys.modules:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True, use_metadata_server=False)

Mounted at /content/drive


In [32]:
if "google.colab" in sys.modules:
    import requests
    # Save datagenerators as file to colab working directory
    # If you are using GitHub, make sure you get the "Raw" version of the code
    url = 'https://raw.githubusercontent.com/karimassi/road-segmentation/main/src/training.py?token=AFUMR6FMXJI4DQHDQGBSNEK7YYMNU'
    r = requests.get(url)

    with open('training.py', 'w') as f:
        f.write(r.text)

    url = 'https://raw.githubusercontent.com/karimassi/road-segmentation/main/src/MyDataSet.py?token=AFUMR6FZBJ3XYQUEWMNXVGS7YYQRI'
    r = requests.get(url)
        
    with open('MyDataSet.py', 'w') as f:
        f.write(r.text)
        
    import MyDataSet
    import training 
else:
    from src import training   
    from src import MyDataSet

In [21]:
threshold = 0.25 # percentage of pixels > 1 required to assign a foreground label to a patch

class PatchModel(nn.Module):
    """
    Model that tells if a 16 x 16 RGB (as a 3 x 16 x 16 tensor) correspond to a road (1) or not (0)
    """
    def __init__(self):
        super().__init__()
        
        # 3 channels 16 x 16
        self.conv1 = nn.Conv2d(
            in_channels=3,
            out_channels=10,
            kernel_size=5
        )
        # 10 channels 12 x 12 (12 = 16 - (kernel_size - 1))
        self.pool1 = nn.MaxPool2d(kernel_size=2)
        # 10 channels 6 x 6 (6 = 12 / kernel_size)
        self.conv2 = nn.Conv2d(
            in_channels=10,
            out_channels=20,
            kernel_size=3
        )
        # 20 channels 4 x 4 (4 = 6 - (kernel_size - 1))
        self.pool2 = nn.MaxPool2d(kernel_size=2)
        # 20 channels 2 x 2 (2 = 4 / kernel_size)
        
        self.lin1 = nn.Linear(
            in_features=20 * 2 * 2,
            out_features=10
        )
        self.lin2 = nn.Linear(
            in_features=10,
            out_features=1
        )
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        x = self.relu(self.pool1(self.conv1(x)))
        x = self.relu(self.pool2(self.conv2(x)))
        x = x.view(-1, 20 * 2 * 2)
        x = self.relu(self.lin1(x))
        x = self.sigmoid(self.lin2(x))
        # If we are in testing mode then the output should be either 0 or 1
        if not self.training:
            x = 1 * (x > threshold)
        return x.view(-1)

In [23]:
num_epochs = 10
learning_rate = 1e-3
batch_size = 100

dataset = MyDataSet.PatchedSatImagesDataset(MyDataSet.img_path, MyDataSet.gt_path, threshold)

rotations = [(0, 45), (45, 90), (90, 135), (135, 180), (-45, 0), (-90, -45), (-135, -90), (-180, -135)]
for rotation in rotations:
    dataset += MyDataSet.PatchedSatImagesDataset(MyDataSet.img_path, MyDataSet.gt_path, threshold, transforms.RandomRotation(rotation))

data_len = len(dataset)
train_len = int(data_len * 0.7)
test_len = int(data_len * 0.3)

dataset_train, dataset_test = random_split(dataset, [train_len, test_len])

print(len(dataset_train), len(dataset_test))

dataloader_train = torch.utils.data.DataLoader(
    dataset_train,
    batch_size=batch_size,
    shuffle=True
)

dataloader_test = torch.utils.data.DataLoader(
    dataset_test,
    batch_size=batch_size,
    shuffle=True
)

# Train the logistic regression model with the Adam optimizer
criterion = torch.nn.MSELoss()
model = PatchModel().to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
train(model, criterion, dataloader_train, dataloader_test, optimizer, num_epochs)


393750 168750
Starting training
Epoch  1 | Test accuracy : 0.78519 | Test F1 : 0.61645
Epoch  2 | Test accuracy : 0.85148 | Test F1 : 0.69238
Epoch  3 | Test accuracy : 0.88124 | Test F1 : 0.73307
Epoch  4 | Test accuracy : 0.89133 | Test F1 : 0.75651
Epoch  5 | Test accuracy : 0.84621 | Test F1 : 0.70491
Epoch  6 | Test accuracy : 0.903 | Test F1 : 0.77572
Epoch  7 | Test accuracy : 0.9111 | Test F1 : 0.79106
Epoch  8 | Test accuracy : 0.8453 | Test F1 : 0.70936
Epoch  9 | Test accuracy : 0.91277 | Test F1 : 0.78106
Epoch  10 | Test accuracy : 0.91624 | Test F1 : 0.80763


In [37]:
import pickle 
pickle.dump(model, open( "model.pkl", "wb" ) )
torch.save(model, 'model_2.pkl')

In [ ]:
submission = MyDataSet.PatchedTestSatImagesDataset(MyDataSet.test_path)
dataloader_submission = torch.utils.data.DataLoader(
    submission, 
    batch_size=1,
    shuffle=False
)
with open('submission.csv', 'w') as f:
    f.write('id,prediction\n')
    model.eval()
    for img_id, X in dataloader_submission:
        X = X.to(device)
        Y = model(X).item()
        f.write(f'{img_id},{Y}')        